In [104]:
import cv2
import numpy as np
from keras.models import load_model
import mediapipe as mp

In [105]:
mp_hands = mp.solutions.hands

In [106]:
def get_hand_image(image):
    width = image.shape[1]
    height = image.shape[0]
    with mp_hands.Hands(min_detection_confidence=0.3, min_tracking_confidence=0.3) as hands:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                landmarks = np.array([[lm.x * width, lm.y * height] for lm in hand_landmarks.landmark])
                hand_bbox = cv2.boundingRect(landmarks.astype(np.float32))
                # Check if the hand bounding box is valid
                if hand_bbox[2] > 0 and hand_bbox[3] > 0:
                    # Extract hand image
                    hand_image = image[hand_bbox[1]:hand_bbox[1] + hand_bbox[3], hand_bbox[0]:hand_bbox[0] + hand_bbox[2]]
                    # Check if hand_image is not empty
                    if hand_image.size != 0:
                        # Resize hand image
                        hand_image = cv2.resize(hand_image, (64, 64))
                        return hand_image
                    else:
                        print("Error: Empty hand image")
                else:
                    print("Error: Invalid hand bounding box")
        else:
            print("Error: No hand detected")
    # Return None if no hand image is found
    return None

In [107]:
def preprocess_frame(frame):
    frame = get_hand_image(frame)
    if frame is None:
        return None
    frame = frame / 255.0  # Normalize pixel values to [0, 1]
    return frame

In [108]:
frame_sequence = []


In [109]:
model =  load_model('30frame_best_model.h5')

In [110]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()  # Read frame from the video capture
    if not ret:
        break  # Break the loop if no frame is read

    # Preprocess frame
    preprocessed_frame = preprocess_frame(frame)

    # Add preprocessed frame to frame sequence
    if preprocessed_frame is not None:
        frame_sequence.append(preprocessed_frame)

    # Keep only the last 30 frames in the sequence
    if len(frame_sequence) > 30:
        frame_sequence = frame_sequence[-30:]

    # Once we have enough frames, process the sequence
    if len(frame_sequence) == 30:
        # Concatenate frames along a new axis to create a sequence
        sequence_input = np.array(frame_sequence)[np.newaxis, ...]
        print(sequence_input)
        prediction_prob = model.predict(sequence_input)
        prediction = np.argmax(prediction_prob, axis=1)[0]
        if prediction == 0:
            prediction = 'Correct'
        else:
            prediction = 'Incorrect'
        cv2.putText(frame, "Prob: {}".format(prediction,prediction_prob), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Frame', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close OpenCV windows
cap.release()
cv2.destroyAllWindows()

Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand d